# Neural Network model of preterm birth prediction

# Package

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf

# Data

In [27]:
alldata = pd.read_csv("alldata.csv")
alldata.head()

,Unnamed: 0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,...,g_32827,g_32828,g_32829,g_32830,SampleID,GA,Batch,Set,Train,Platform
0,1,6.062215,3.796484,5.849338,3.567779,6.166815,4.443027,5.836522,6.330018,4.922339,...,8.972873,10.440245,12.101476,13.695705,Tarca_001_P1A01,11.0,1,PRB_HTA,1,HTA20
1,2,6.125023,3.805305,6.191562,3.452524,5.678373,4.773199,6.143398,5.601745,4.711765,...,9.376194,10.845176,12.370891,13.635522,Tarca_003_P1A03,NaN,1,PRB_HTA,0,HTA20
2,3,5.875502,3.450245,6.550525,3.316134,6.185059,4.393488,5.898364,6.137984,4.628124,...,8.843612,10.493416,12.295786,13.616688,Tarca_004_P1A04,32.6,1,PRB_HTA,1,HTA20
3,4,6.126131,3.628411,6.421877,3.432451,5.633757,4.623783,6.019792,5.787502,4.796283,...,9.191471,10.879879,12.249936,13.524328,Tarca_005_P1A05,30.6,1,PRB_HTA,1,HTA20
4,5,6.146466,3.446812,6.260962,3.477162,5.313198,4.422651,6.407699,5.830437,4.726488,...,9.247768,10.754316,12.245458,13.509353,Tarca_006_P1A06,NaN,1,PRB_HTA,0,HTA20


In [7]:
# X_train_df = alldata.loc[alldata['Train'] == 1, 'g_1':'g_32830']
# y_train_df = alldata.loc[alldata['Train'] == 1, 'GA']
# X_test_df = alldata.loc[alldata['Train'] == 0, 'g_1':'g_32830']

367

# Preprocessing

In [32]:
COLUMNS = alldata.columns[1:32831]
COLUMNS_train = COLUMNS.insert(0, 'GA')
COLUMNS_test = COLUMNS

In [33]:
train = alldata.loc[alldata['Train'] == 1, COLUMNS_train]
test = alldata.loc[alldata['Train'] == 0, COLUMNS_test]

In [34]:
n_train = train.shape[0]

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('GA')

mat_train = np.matrix(train)
mat_test = np.matrix(test)
mat_new = np.matrix(train.drop('GA', axis = 1))
mat_y = np.array(train.GA).reshape((n_train, 1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train), columns = col_train)
test = pd.DataFrame(prepro_test.transform(mat_test), columns = col_train_bis)

train.head()

,GA,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,...,g_32821,g_32822,g_32823,g_32824,g_32825,g_32826,g_32827,g_32828,g_32829,g_32830
0,0.060897,0.414799,0.551520,0.143515,0.415253,0.710906,0.409695,0.194868,0.825319,0.679118,...,0.080474,0.045069,0.239724,0.279605,0.090182,0.206483,0.376036,0.270790,0.375436,0.663293
1,0.753205,0.229841,0.131113,0.931087,0.159929,0.721855,0.376144,0.253598,0.679615,0.373204,...,0.437876,0.491024,0.279387,0.259794,0.166607,0.674196,0.326170,0.295013,0.497897,0.569575
2,0.689103,0.478114,0.347445,0.786591,0.277946,0.390990,0.532112,0.368918,0.413692,0.548050,...,0.718993,0.744397,0.392527,0.281956,0.362166,0.000000,0.460366,0.471073,0.469001,0.460033
3,0.544872,0.486459,0.206491,0.486714,0.486642,0.099551,0.561675,0.334930,0.265410,0.549104,...,0.344491,0.392312,0.126068,0.360957,0.350637,0.121205,0.323854,0.264671,0.383532,0.451854
4,0.432692,0.639981,0.274327,0.703695,0.244051,0.395123,0.769862,0.459255,0.454115,0.675790,...,0.560711,0.660547,0.194908,0.321883,0.226707,0.247913,0.247627,0.233851,0.283125,0.147626


To use TensorFlow, need to transform data

In [36]:
# list of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = 'GA'

# Columns for tensorflow
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

# training set and prediction set with features to predict
training_set = train[COLUMNS]
prediction_set = train.GA

# train and test
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES], 
                                                   prediction_set,
                                                   test_size = 0.33,
                                                   random_state = 42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
print(training_set.head())

# training for submission
training_sub = training_set[col_train]


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

          g_1       g_2       g_3       g_4       g_5       g_6       g_7  \
31   0.449572  0.434498  0.608204  0.366742  0.269445  0.583950  0.517757   
36   0.380233  0.108865  0.955788  0.230168  0.594310  0.605353  0.443546   
286  0.423252  0.336646  0.458898  0.367393  0.338555  0.627292  0.315661   
181  0.703718  0.398473  0.587536  0.232097  0.627203  0.526318  0.440456   
324  0.568930  0.458742  0.500383  0.267163  0.194856  0.609781  0.820340   

          g_8       g_9      g_10    ...      g_32822   g_32823   g_32824  \
31   0.369939  0.397291  0.525432    ...     0.577337  0.947747  0.797428   
36   0.831193  0.352485  0.533070    ...     0.560260  0.572935  0.628506   
286  0.622776  0.592889  0.304886    ...     0.688651  0.609609  0.765

In [37]:
# same thing for test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,g_10,...,g_32822,g_32823,g_32824,g_32825,g_32826,g_32827,g_32828,g_32829,g_32830,GA
193,1.000000,0.542018,0.766475,0.613756,0.130965,0.900691,0.621182,0.170812,0.866174,0.701076,...,0.728252,0.540747,0.426365,0.555813,0.537314,0.402248,0.439340,0.306938,0.131591,0.679487
33,0.531161,0.188048,0.596096,0.288369,0.394546,0.517247,0.182765,0.587824,0.465758,0.373441,...,0.369066,0.572673,0.567947,0.593022,0.573346,0.517036,0.500367,0.638846,0.698745,0.009615
15,0.318686,0.359582,0.403324,0.341275,0.791831,0.491907,0.343877,0.760651,0.299314,0.192798,...,0.361683,0.931356,0.818309,0.861305,0.826920,0.524454,0.546194,0.568070,0.751403,0.711538
311,0.471356,0.395336,0.660133,0.311523,0.596514,0.681251,0.608307,0.499232,0.727575,0.493111,...,0.708816,0.632840,0.718387,0.649507,0.619193,0.450858,0.529716,0.539282,0.392565,0.608974
57,0.519277,0.490927,0.395352,0.304773,0.220300,0.490224,0.257738,0.477952,0.334281,0.336528,...,0.451036,0.501599,0.444616,0.541656,0.435237,0.345737,0.314437,0.329119,0.426192,0.769231


# Deep Neural Network

In [39]:
# model
tf.logging.set_verbosity(tf.logging.ERROR)
regressor = tf.contrib.learn.DNNRegressor(feature_columns = feature_cols,
                                         activation_fn = tf.nn.relu,
                                         hidden_units = [200, 100, 50, 25, 12])

In [40]:
# reset index
training_set.reset_index(drop = True, inplace = True)